https://github.com/traja-team/traja?tab=readme-ov-file

remove the negative value sfrom the plot!!!

In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches

# ---------- read log.csv ----------
x_vals = []
y_vals = []

with open("log.csv", "r") as f:
    lines = f.readlines()

# skip header
for line in lines[1:]:
    m = re.search(
        r"X:\s*([-+]?\d*\.\d+|\d+)\s*cm,\s*Y:\s*([-+]?\d*\.\d+|\d+)\s*cm",
        line
    )
    if m:
        x_vals.append(float(m.group(1)))
        y_vals.append(float(m.group(2)))
    else:
        print("Could not parse line:", line.strip())

x = np.array(x_vals)
y = np.array(y_vals)

# ---------- filter function ----------
def filter_xy_nonnegative(x, y, threshold=0.0):
    """
    Remove points where x < threshold or y < threshold.
    Returns filtered x, y, and the boolean mask.
    """
    x = np.asarray(x)
    y = np.asarray(y)
    mask = (x >= threshold) & (y >= threshold)
    x_f = x[mask]
    y_f = y[mask]
    assert x_f.shape == y_f.shape
    return x_f, y_f, mask

# apply filter: remove all negative entries (x<0 or y<0)
x, y, mask = filter_xy_nonnegative(x, y, threshold=0.0)
t = np.arange(len(x_vals))[mask]  # keep time aligned with filtered points

# ---------- helpers ----------
def stylize_axes(ax):
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_aspect("equal")

def plot_trajectory_2d(x, y, t=None, ax=None, cmap="viridis"):
    x = np.asarray(x)
    y = np.asarray(y)
    n = len(x)
    if t is None:
        t = np.arange(n)

    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))
    else:
        fig = ax.figure

    # background path (grey line)
    verts = np.column_stack([x, y])
    codes = [Path.MOVETO] + [Path.LINETO] * (n - 1)
    path = Path(verts, codes)
    patch = patches.PathPatch(
        path, edgecolor="#999999", facecolor="none", lw=3, alpha=0.3
    )
    ax.add_patch(patch)

    # colored scatter
    sc = ax.scatter(
        x, y,
        c=t,
        s=5,
        cmap=cmap,
        alpha=0.8,
    )

    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(y.min(), y.max())
    ax.set_xlabel("X [cm]")
    ax.set_ylabel("Y [cm]")
    stylize_axes(ax)

    cbar = fig.colorbar(sc, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label("time / frame")

    fig.tight_layout()
    return ax, sc, cbar

# ---------- main call ----------
ax, sc, cbar = plot_trajectory_2d(x, y, t)
plt.title("2D Trajectory (filtered nonnegative)")
plt.savefig("trajectory_2d.png", dpi=300)
plt.close()

print("Saved: trajectory_2d.png")


Saved: trajectory_2d.png
